# DB connection

In [ ]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
import pandas as pd
from datetime import datetime

# Загрузка переменных окружения
load_dotenv()

# Подключение к БД
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
dbname = os.getenv("DB_NAME")

db_url = f"postgresql://{user}:{password}@{host}:{port}/{dbname}"
engine = create_engine(db_url)

# Пути
os.makedirs("data", exist_ok=True)
csv_path = "data/incidents.csv"
timestamp_file = "data/last_timestamp.txt"

# Получение последнего timestamp
if os.path.exists(timestamp_file):
    with open(timestamp_file, "r") as f:
        last_timestamp = f.read().strip()
        print(f"Последний timestamp из файла: {last_timestamp}")
    # Если нужно, приводим last_timestamp к нужному формату:
    # last_timestamp = pd.to_datetime(last_timestamp)
else:
    last_timestamp = None
    print("Файл с timestamp не найден, загружаем с начала")

batch_size = 1000
total_rows_fetched = 0

while True:
    if last_timestamp:
        # Если last_timestamp уже задан, берем строки после этого timestamp
        query = f"""
        SELECT id, timestamp, data, business_status
        FROM public.incident
        WHERE timestamp > '{last_timestamp}'
        ORDER BY timestamp
        LIMIT {batch_size}
        """
    else:
        # Если нет last_timestamp, пусть, к примеру, берем всё после id>10000
        # Или, если хотим именно по timestamp — можно задать какую-то нижнюю границу
        query = f"""
        SELECT id, timestamp, data, business_status
        FROM public.incident
        WHERE id > 10000
        ORDER BY timestamp
        LIMIT {batch_size}
        """

    batch_df = pd.read_sql_query(query, engine)

    rows = len(batch_df)
    if rows == 0:
        print("Новых данных не найдено — выходим из цикла.")
        break
    
    # Если csv еще не существует — пишем с заголовками, иначе — без них
    if not os.path.exists(csv_path):
        batch_df.to_csv(csv_path, index=False, mode='w', header=True)
    else:
        batch_df.to_csv(csv_path, index=False, mode='a', header=False)
    
    # Считаем общее кол-во
    total_rows_fetched += rows

    # Обновляем last_timestamp (максимальный из текущего батча)
    new_last_timestamp = batch_df["timestamp"].max()
    with open(timestamp_file, "w") as f:
        f.write(str(new_last_timestamp))

    print(f"Выгрузили батч из {rows} строк. Новый last_timestamp={new_last_timestamp}")
    
    # Для следующей итерации
    last_timestamp = new_last_timestamp

print(f"Всего выгружено: {total_rows_fetched} строк")


In [ ]:
df = pd.read_csv(csv_path)

In [ ]:
df.tail()